In [3]:
from langchain_google_genai import GoogleGenerativeAI

In [5]:

!pip install langchain-google-genai

import os
from dotenv import load_dotenv
load_dotenv() 

#llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=os.environ["GOOGLE_API_KEY"])
llm = GoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
print(llm("write few lines on Generative AI"))

C:\Users\Admin\AppData\Local\Temp\ipykernel_15068\266188152.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm("write few lines on Generative AI"))


Of course, here are a few ways to describe Generative AI in a few lines:

**Simple & Direct**

Generative AI is a type of artificial intelligence that can create new, original content, such as text, images, music, and code. It learns from vast amounts of existing data and then uses that knowledge to produce something entirely new, yet similar in style or structure.

***

**More Evocative**

Think of Generative AI as a creative partner that has studied nearly all of human creativity. It doesn't just analyze information; it synthesizes it to dream up new essays, design stunning visuals, or compose original melodies, revolutionizing how we create.

***

**In Bullet Points**

*   **Creates New Content:** It produces original text, images, and audio, rather than just analyzing existing data.
*   **Learns from Patterns:** It's trained on massive datasets to understand the underlying structures and styles.
*   **A Powerful Tool:** It acts as a collaborator for artists, writers, and programmer

#### Connect with database and ask some basic questions

In [9]:

#from langchain_community.utilities import SQLDatabase
#from langchain_experimental.sql import SQLDatabaseChain
!pip install langchain
!pip install -U langchain-community

#from langchain.chains import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from langchain.utilities import SQLDatabase


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 2.8 MB/s eta 0:00:01
   -------------------- ------------------- 1.3/2.5 MB 3.0 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.5 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.30
    Uninstalling langchain-community-0.3.30:
      Successfully uninstalled langchain-community-0.3.30


In [12]:
!pip install pymysql

db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)DEFAULT CHARSET=utf8mb4 ENGINE=MyISAM COLLATE utf8mb4_0900_ai_ci

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [14]:
#Convert question to SQL query
from langchain.chains import create_sql_query_chain
import re
chain = create_sql_query_chain(llm, db)

def clean_query(response_text):
    """
    Cleans and extracts the SQL query from Gemini's response.
    Handles cases like:
    - 'Question: ...'
    - 'SQLQuery: SELECT ...'
    - Code blocks (```sql ... ```)
    - Extra text before/after SQL
    """
    text = response_text.strip()

    # Remove code block markers like ```sql and ```
    text = re.sub(r"```sql", "", text, flags=re.IGNORECASE)
    text = re.sub(r"```", "", text)

    # Remove leading labels like 'Question:' or 'SQLQuery:'
    text = re.sub(r"(?i)(question:|sqlquery:)", "", text)

    # Extract the SQL statement (starts with SELECT/INSERT/UPDATE/DELETE)
    match = re.search(r"(SELECT|INSERT|UPDATE|DELETE).*", text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(0).strip()
    else:
        return text.strip()
        
response = chain.invoke({"question": "How many customers are there"})
response

'Question: How many customers are there\nSQLQuery: SELECT count(DISTINCT `CustomerID`) FROM sales_tb'

In [16]:
# cleaned_query = response.strip('```sql\n').strip('\n```')
# print(cleaned_query)
# Clean the query text
cleaned_query = clean_query(str(response))
print("\n✅ Cleaned SQL Query:\n", cleaned_query)


✅ Cleaned SQL Query:
 SELECT count(DISTINCT `CustomerID`) FROM sales_tb


In [23]:
# Execute the cleaned query
result = db.run(cleaned_query)
print(result)

[(29,)]


In [30]:
# chain = create_sql_query_chain(llm, db)
# def execute_query(question):
#     try:
#         # Generate SQL query from question
#         response = chain.invoke({"question": question})
#         print(response)
#         print("###################################################")
#         # Strip the formatting markers from the response
#         cleaned_query = response.strip('```sql\n').strip('\n```')
#         print(cleaned_query)
#         print("###################################################")        
#         # Execute the cleaned query
#         result = db.run(cleaned_query)
#         print("###################################################")        
#         # Display the result
#         print(result)
#     except ProgrammingError as e:
#         print(f"An error occurred: {e}")
import re
from sqlalchemy.exc import ProgrammingError

chain = create_sql_query_chain(llm, db)

def clean_query(response_text):
    """
    Cleans and extracts only the SQL query from Gemini's response.
    Removes 'Question:', 'SQLQuery:', code blocks, etc.
    """
    text = str(response_text).strip()

    # Remove markdown-style SQL code blocks
    text = re.sub(r"```sql", "", text, flags=re.IGNORECASE)
    text = re.sub(r"```", "", text)

    # Remove helper labels like "Question:" or "SQLQuery:"
    text = re.sub(r"(?i)(question:|sqlquery:)", "", text)

    # Extract the actual SQL statement
    match = re.search(r"(SELECT|INSERT|UPDATE|DELETE).*", text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(0).strip()
    else:
        return text.strip()


def execute_query(question):
    try:
        # Generate SQL query from the LLM
        response = chain.invoke({"question": question})
        print("LLM raw response:")
        print(response)
        print("###################################################")

        # Clean the SQL query
        cleaned_query = clean_query(response)
        print("Cleaned SQL Query:")
        print(cleaned_query)
        print("###################################################")

        # Execute the cleaned SQL query
        result = db.run(cleaned_query)
        print("Query Result:")
        print(result)
        print("###################################################")
        return cleaned_query, result

    except ProgrammingError as e:
        print(f"❌ SQL Error: {e}")
        return None, None
    except Exception as e:
        print(f"❌ General Error: {e}")
        return None, None



In [32]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

LLM raw response:
Question: How many unique customers are there for each product category
SQLQuery: SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `num_customers` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `num_customers` DESC LIMIT 5
###################################################
Cleaned SQL Query:
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `num_customers` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `num_customers` DESC LIMIT 5
###################################################
Query Result:
[('Clothing', 13), ('Beauty', 8), ('Electronics', 8)]
###################################################


('SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `num_customers` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `num_customers` DESC LIMIT 5',
 "[('Clothing', 13), ('Beauty', 8), ('Electronics', 8)]")

In [34]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

LLM raw response:
SQLQuery:
```sql
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` FROM sales_tb GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5;
```
###################################################
Cleaned SQL Query:
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` FROM sales_tb GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5;
###################################################
Query Result:
[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]
###################################################


('SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales` FROM sales_tb GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5;',
 "[('Electronics', Decimal('5310.00')), ('Clothing', Decimal('5040.00')), ('Beauty', Decimal('1455.00'))]")

In [36]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

LLM raw response:
SQLQuery: SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender`
###################################################
Cleaned SQL Query:
SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender`
###################################################
Query Result:
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]
###################################################


('SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender`',
 "[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]")

In [38]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

LLM raw response:
Question: identify the top spending customers based on their total amount spent.
SQLQuery: SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM sales_tb GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5
###################################################
Cleaned SQL Query:
SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM sales_tb GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5
###################################################
Query Result:
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]
###################################################


('SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM sales_tb GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5',
 "[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]")

In [44]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

LLM raw response:
Question: counts the number of transactions made each month.
SQLQuery: SELECT DATE_FORMAT(`Date`, '%Y-%m') AS `month`, COUNT(`TransactionID`) AS `number_of_transactions` FROM `sales_tb` GROUP BY `month` ORDER BY `month` LIMIT 5
###################################################
Cleaned SQL Query:
SELECT DATE_FORMAT(`Date`, '%Y-%m') AS `month`, COUNT(`TransactionID`) AS `number_of_transactions` FROM `sales_tb` GROUP BY `month` ORDER BY `month` LIMIT 5
###################################################
Query Result:
[('2023-01', 4), ('2023-02', 4), ('2023-03', 1), ('2023-04', 5), ('2023-05', 2)]
###################################################


("SELECT DATE_FORMAT(`Date`, '%Y-%m') AS `month`, COUNT(`TransactionID`) AS `number_of_transactions` FROM `sales_tb` GROUP BY `month` ORDER BY `month` LIMIT 5",
 "[('2023-01', 4), ('2023-02', 4), ('2023-03', 1), ('2023-04', 5), ('2023-05', 2)]")

In [42]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

LLM raw response:
SQLQuery: SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales`, AVG(`PriceperUnit`) AS `AveragePricePerUnit` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5
###################################################
Cleaned SQL Query:
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales`, AVG(`PriceperUnit`) AS `AveragePricePerUnit` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5
###################################################
Query Result:
[('Electronics', Decimal('5310.00'), Decimal('238.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Beauty', Decimal('1455.00'), Decimal('153.750000'))]
###################################################


('SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSales`, AVG(`PriceperUnit`) AS `AveragePricePerUnit` FROM `sales_tb` GROUP BY `ProductCategory` ORDER BY `TotalSales` DESC LIMIT 5',
 "[('Electronics', Decimal('5310.00'), Decimal('238.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Beauty', Decimal('1455.00'), Decimal('153.750000'))]")